#  Must run file 14* first, which sets up MongoDB

In [ ]:

#  Hello World for MongoDB ..   
#
#  Sample_Mflix     https://www.mongodb.com/docs/atlas/sample-data/sample-mflix/

import pymongo
from   pymongo import MongoClient
   #
from bson.objectid import ObjectId

   ###

l_cn_m = MongoClient("localhost:7777")

l_db_m = l_cn_m.my_db_m

print("--")


# MongoDB demonstration database for movies ..

In [ ]:

#  MongoDB movies database only exists after a series of mongorestore commands ..

      mongorestore --host=localhost --port=7777 -d my_db_m -c users       users.bson
      mongorestore --host=localhost --port=7777 -d my_db_m -c movies      movies.bson
      mongorestore --host=localhost --port=7777 -d my_db_m -c comments    comments.bson
      mongorestore --host=localhost --port=7777 -d my_db_m -c theaters    theaters.bson
      mongorestore --host=localhost --port=7777 -d my_db_m -c sessions    sessions.bson


In [ ]:

#  Count each collection ..

display(l_db_m.users.count_documents({})   )
display(l_db_m.movies.count_documents({})  )
display(l_db_m.comments.count_documents({}))
display(l_db_m.sessions.count_documents({}))
display(l_db_m.theaters.count_documents({}))


In [ ]:

#  Get the _id of a Movie

l_result = list (l_db_m.movies.find({}).limit(1))
   #
for l_each in l_result:
   print(l_each["_id"])

print("--")

#  573a1390f29313caabcd4135      From fetch above
#  573a1390f29313caabcd418c      From Comments below


In [ ]:

#  Fetch using the _id from above

l_result = list (l_db_m.movies.find({ "_id": ObjectId("573a1390f29313caabcd4135") }) )
# l_result = list (l_db_m.movies.find({ "_id": ObjectId("573a1390f29313caabcd418c") }) )
   #
for l_each in l_result:
   print(l_each["_id"])

print("--")


In [ ]:
# #######################################################

In [ ]:

#  Intersection between Comments and Movies, then Comments and Users ..
#
#  (We use this query to create the Edge record between Users and Movies.)
#
#    50,304  Comments
#    23,539  Movies
#
#    41,080  Comments have a movie_id found in Movies
#       then
#    41,006  Comments have an email found in Users


l_result = list ( l_db_m.comments.aggregate( [
   
   #  Start with Comments --> Movies
   #
   {
   "$lookup" :
      {
      "localField"   : "movie_id",
         #
      "from"         : "movies",
      "foreignField" : "_id",
      "as"           : "ddd"
      }
   },
   {
   "$project" :
      {
      "_id"     : 0,                                      #  PK from Comments, don't need this
         #
      "movie_id": 1,                                      #  FK into Movies
      "email"   : 1,                                      #  FK into Users
         #
      "name"    : 1,                                      #  What we get from Comments
      "text"    : 1,
         #
      "ddd_size": {"$size": "$ddd"}                       #  These are effectively outer joins, filter out where no match existed
      }
   } ,
   {
   "$match":                                              #  41,080  rows will pass this point  (Comments --> Movies)
      {
      "ddd_size": {"$gt": 0}
      }
   },
    
   #  Move now to Comments --> Users
   # 
   {
   "$lookup" :
      {
      "localField"   : "email",
         #
      "from"         : "users",
      "foreignField" : "email",
      "as"           : "eee"
      }
   },
   {
   "$project" :
      {
      "_id"     : 0,                                      #  PK from Comments, don't need this
         #
      "movie_id": 1,                                      #  FK into Movies
      "email"   : 1,                                      #  FK into Users
         #
      "name"    : 1,                                      #  What we get from Comments
      "text"    : 1,
         #
      "eee_size": {"$size": "$eee"}                       #  These are effectively outer joins, filter out where no match existed
      }
   } ,
   {
   "$match":                                              #  41,006  rows will pass this point  (Comments --> Users)
      {
      "eee_size": {"$gt": 0}
      }
   },
    
   #  Final project
   #
   {
   "$project" :
      {
      "_id"     : 0,                                      #  PK from Comments, don't need this
         #
      "movie_id": 1,                                      #  FK into Movies
      "email"   : 1,                                      #  FK into Users
         #
      "name"    : 1,                                      #  What we get from Comments
      "text"    : 1,
      }
   } ,
    
#  {
#  "$limit" : 5
#  }
    
#  {
#  "$count" : "total_count"
#  }
    
    
] ) )
    
    
l_cntr = 0
    
for l_each in l_result:
   l_movie_id = str(l_each["movie_id"])
   l_email    =     l_each["email"   ]
   l_name     =     l_each["name"    ]
   l_text     =     l_each["text"    ][:48]
      #
   l_cntr += 1
   if (l_cntr < 10):
      print("%-24s  %-28s  %-18s  %-48s" % (l_movie_id, l_email, l_name, l_text[:48]))

print("--")


In [ ]:

#  Write the Edge data produced above to a CSV file ..

l_outputfile = "./08_CsvData/30_Edge_Commented.txt"
   #
l_header     = "movie_id,email,name,text"

with open(l_outputfile, 'w') as l_file:
   l_file.write(l_header)
       #
   for l_each in l_result:
         #
      l_movie_id = "ObjectId(" + str(l_each["movie_id"]) + ")"
      l_email    =     l_each["email"   ]
      l_name     =     l_each["name"    ]
      l_text     =     l_each["text"    ].replace('\n',' ')
         #
      l_str      = l_movie_id + "," + l_email + "," + l_name + "," + l_text 
         #
      l_file.write('\n')
      l_file.write(l_str)
        
print("--")
        